# "RecoTour III: Variational Autoencoders for Collaborative Filtering with Mxnet and Pytorch"
> An deep dive into the use of Variational Autoencoders for Collaborative Filtering

- author: Javier Rodriguez
- toc: true 
- badges: true
- comments: true
- categories: [jupyter]

This post and the code here are part of a larger repo called [RecoTour](https://github.com/jrzaurin/RecoTour), where I normally explore and implement some recommendation algorithms that I consider interesting and/or useful (see [RecoTour](https://medium.com/datadriveninvestor/recotour-a-tour-through-recommendation-algorithms-in-python-52d780628ab9) and [RecoTourII](https://towardsdatascience.com/recotour-ii-neural-recommendation-algorithms-49733938d56e)). In every directory, I have included a `README` file and a series of explanatory notebooks that I hope help explaining the code. I keep adding algorithms from time to time, so stay tunned if you are interested. 

As always, let me first acknowledge the relevant people that did the hard work. This post and the companion repo are based on the paper “Variational Autoencoders for Collaborative Filtering” [1]. The code in that repo is partially inspired in the implementation by [Younggyo Seo](https://github.com/younggyoseo/vae-cf-pytorch). I have adapted the code to my coding preferences and added some options and flexibility to run multiple experiment. The explanation of the algorithm in Section ? is based on the following sources:

1. [EM Demystified: An Expectation-Maximization Tutorial]() [2]
2. [Auto-Encoding Variational Bayes]() [3]
3. [Variational Autoencoders with Gluon]()

as well as of course the [paper](https://arxiv.org/pdf/1802.05814.pdf) in which this post is based. The datasets I have used for this experiment are the MovieLens-20M dataset and the Amazon dataset [4] [5]. The former is mainly use so I can make sure I am obtaining consistent results to those obtained in the [paper](https://arxiv.org/pdf/1802.05814.pdf). 

This post is structured as follows: Section 1 describes the data preparation while Section 2 intends to be a relatively soft explanation of the maths behind the algorithm. Sections 3 focuses on how to run the algorithm while Section 4 is a summary of the results. Finally, Section 5 is a brief description of what I intend to add next to the RecoTour repo.  